In [1]:
import os
import sys
import requests
import time
import gc
import pandas as pd

from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
from dotenv import load_dotenv

sys.path.append("../")

from scripts import preprocessing

load_dotenv()
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not
try:
    pc_dpi = int(os.getenv('DPI'))
except TypeError:
    pc_dpi = 100
if pc_dpi is None:
    pc_dpi = 100


/var/folders/3s/s8sp6jwn6qs02jfxbgjc7c_40000gn/T/ipykernel_4844/2473611209.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Purpose

The primary objectives of this project include:

- **Feasibility of Database Recreation:** Evaluating the practicality of recreating a comprehensive meteorite database by aggregating data from a single, extensive request.
- **Establishment of Initial Search Step:** Setting up the foundational step in the meteorite property search process, where one request retrieves all necessary URLs, types, names, places of fall, etc. This is aimed at reducing the overall impact on server resources compared to performing a new search request each time a user initiates a search. Instead, the user will search through a JSON file extracted from the webpage, not the page itself.
- **Reproducibility:** Ensuring the process can be automatically updated with tools like GitHub Actions or other automation tools configured similarly to CRON jobs. The goal is for this notebook to pave the way for a script that facilitates the "get-links" pipeline, making the dataset self-updating and reliable over time.
- **Handling Large Requests:** Recognizing that the request involves parsing over 4 million lines of HTML, which introduces potential for errors such as server timeouts or connection interruptions. Implementing checks is crucial, for example, verifying the end of the file for a closing body tag or closing HTML tag (`</html>`, `</body>`) to ensure the complete dataset is captured.
- **Export Format:** The data should be exported as a JSON file with fields that are straightforward to navigate and search, making the dataset accessible and usable for various applications.

This structured approach aims to minimize the load on the source server, streamline the data collection process, and ensure the sustainability and usability of the meteorite database.


In [2]:
t_zero = time.perf_counter()

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0",
}

behemoth = """https://www.lpi.usra.edu/meteor/metbull.php?sea=%2A&sfor=names&ants=\
&nwas=&falls=&valids=&stype=contains&lrec=100000&map=ge&browse=&country=\
All&srt=name&categ=All&mblist=All&rect=\
&phot=&strewn=&snew=0&pnt=Normal%20table&dr=&page=1"""

r = requests.get(url=behemoth, headers=headers,)

soup = BeautifulSoup(r.content, "html.parser")

t_end = time.perf_counter()

print(f"request time : {round(t_end - t_zero, 4)}")


request time : 57.0998


### Let's test that it has a closing body tag and/or closing html tag :

(Let's use the tail of the html because its kindof a large file)

In [3]:
def check_closing_tags_bytes(html_bytes: bytes) -> tuple:
    """
    Check the last bytes of the HTML content for </body> and </html> tags.
    This function is to be launched on the request.content bytes object.
    
    Args:
        html_bytes (bytes): The HTML content as a bytes object.
    
    Returns:
        tuple: A tuple containing two booleans indicating the presence of </body> and </html> tags.
    """

    tail_content = html_bytes[-100:].decode("utf-8", errors="ignore").lower()
    
    closing_body_tag = "</body>" in tail_content
    closing_html_tag = "</html>" in tail_content

    return closing_body_tag, closing_html_tag



In [4]:
body_tag, html_tag = check_closing_tags_bytes(r.content)

print(f"Closing body tag found : {body_tag}")
print(f"Closing html tag found : {html_tag}")


Closing body tag found : True
Closing html tag found : True


## Selecting the main table (we just need the main table)
- We have confirmed correct EOF (end tags)
- We just need the maintable id of the html doc
- We'll free memory by removing the initial soup (we'll keep the r object just in case)
- We'll force a gc.collect() to improve perfs

In [5]:
# So we want the names, classes, and codes to get the URL
main_table = soup.find("table", id="maintable")

del soup
gc.collect()


0

## Now to extract the informations from the HTML :
- We know :
    - Each meteorite is registered as a span : `("span", class_="mname")` for soup
    - Fields orders : Name|Abbrev|Status|Fall|Year|Place|Type|Mass|MetBull|GoogleEarth|Notes
    - We need fields [0, 4, 5, 6, 7]
    - We can deduce the url of the meteorite by its code, with it we can reform the url


In [6]:
meteorites = main_table.find_all("span", class_="mname")
print(f"There are {meteorites.__len__()} meteorites in the html doc")


There are 81224 meteorites in the html doc


## Lets form a dataframe out of this :

In [7]:
base_url = "https://www.lpi.usra.edu/meteor/metbull.php?"
meteorite_data = []

for row in main_table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > 8:  # Ensuring there are enough cells
        anchor = cells[0].find("a")
        if anchor and "href" in anchor.attrs:
            href = anchor["href"]
            code_start = href.find("code=") + len("code=")
            met_code = href[code_start:].split("&")[0] if "&" in href[code_start:] else href[code_start:]
            met_url = f"{base_url}code={met_code}"
        else:
            met_url = None
        
        # Extracting the required fields
        name = preprocessing.handle_name(cells[0].text.strip())
        year = preprocessing.handle_year(cells[4].text.strip())
        country = preprocessing.handle_country(cells[5].text.strip())
        met_type = preprocessing.handle_types(cells[6].text.strip())
        mass = preprocessing.handle_mass(cells[7].text.strip())

        # Append this meteorite's info as a dict
        meteorite_data.append({
            "name": name,
            "year": year,
            "country": country,
            "type": met_type,
            "mass": mass,
            "URL": met_url
        })

# Convert the list of dicts to a pandas DataFrame
df = pd.DataFrame(meteorite_data)


In [8]:
# handling NA years :
df["year"] = df["year"].astype("Int64")
df["year"] = df["year"].fillna(pd.NA)
# handling NA types
df["type"] = df["type"].replace("Unknown", pd.NA)
df["type"] = df["type"].replace("Unknown", pd.NA)
# handling NA countries
df["country"] = df["country"].replace("Unknown", pd.NA)


## Okay we have a dataset, let's try some simple commands :

In [9]:
display(df[df["type"] == "Iron, IIE-an"])


,name,year,country,type,mass,URL
2735,Arlington,1894,USA,"Iron, IIE-an",8940.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
6545,Barranca Blanca,1855,Chile,"Iron, IIE-an",12000.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
16389,Elephant Moraine 83390,1983,Antarctica,"Iron, IIE-an",15.20,https://www.lpi.usra.edu/meteor/metbull.php?co...
29304,Kavarpura,2006,India,"Iron, IIE-an",6800.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
41745,Netschaëvo,1846,Russia,"Iron, IIE-an",250000.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
67150,Weekeroo Station,1924,Australia,"Iron, IIE-an",94200.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
69429,Yamato 791093,1979,Antarctica,"Iron, IIE-an",4.12,https://www.lpi.usra.edu/meteor/metbull.php?co...


In [10]:
display(df[df["name"].str.lower().str.contains("catalina", na=False)])


,name,year,country,type,mass,URL
7801,Catalina 001,2009,Chile,L3,4680.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
7802,Catalina 002,2010,Chile,LL3,61.1,https://www.lpi.usra.edu/meteor/metbull.php?co...
7803,Catalina 003,1999,Chile,"Iron, IVB",3180.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
7804,Catalina 004,2010,Chile,Mesosiderite,37.5,https://www.lpi.usra.edu/meteor/metbull.php?co...
7805,Catalina 005,2010,Chile,H4,228.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
...,...,...,...,...,...,...
8629,Catalina 836,2023,Chile,H5-6,124.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
8630,Catalina 837,2023,Chile,H5,380.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
8631,Catalina 838,2023,Chile,H5,46.9,https://www.lpi.usra.edu/meteor/metbull.php?co...
8632,Catalina 839,2023,Chile,L6,55.9,https://www.lpi.usra.edu/meteor/metbull.php?co...


In [11]:
display(df[df["type"].str.lower().str.contains("l3", na=False)])


,name,year,country,type,mass,URL
7,Aba Panu,2018,Nigeria,L3,160000.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
108,Acfer 039,1989,Algeria,L3.8,225.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
135,Acfer 066,1990,Algeria,LL3.8-6,517.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
149,Acfer 080,1990,Algeria,L3.9,574.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
171,Acfer 102,1990,Algeria,L3-5,346.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
...,...,...,...,...,...,...
81062,Yambo no 2,1975,Congo - Dem. Rep.,L3,3.2,https://www.lpi.usra.edu/meteor/metbull.php?co...
81084,Yazd 002,2018,Iran,LL3,7000.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
81176,Zagora 010,2021,Morocco,L3,480.0,https://www.lpi.usra.edu/meteor/metbull.php?co...
81191,Įdár nad Sázavou,2014,Czech Republic,L3,45.0,https://www.lpi.usra.edu/meteor/metbull.php?co...


In [12]:
display(df[df["country"].str.lower().str.contains("france", na=False)])


,name,year,country,type,mass,URL
62,Abbans-Dessous,2015,France,LL6,3.12,https://www.lpi.usra.edu/meteor/metbull.php?co...
520,Agen,1814,France,H5,30000.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
562,Aire-sur-la-Lys,1769,France,<NA>,6800.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
670,Alais,1806,France,CI1,6000.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
735,Alby sur Chéran,2002,France,Eucrite-mmict,252.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
...,...,...,...,...,...,...
65465,Sologne,1860,France,H5,54.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
65501,Saint-Séverin,1966,France,LL6,271000.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
66216,Toulouse,1812,France,H6,1030.00,https://www.lpi.usra.edu/meteor/metbull.php?co...
67062,Villedieu,1890,France,H4,14000.00,https://www.lpi.usra.edu/meteor/metbull.php?co...


In [16]:
display(df.dtypes)


name        object
year         Int64
country     object
type        object
mass       float64
URL         object
dtype: object